In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl
# 기호 연산 기능 추가
# Add symbolic operation capability
import sympy as sy

다음과 같은 트러스를 생각해 보자.<br>
Let's think about a truss as follows.<br>
(ref: "[Application of system of linear equations](https://www.chegg.com/homework-help/questions-and-answers/application-system-linear-equations-sure-work-matlab-problem-figure-1-shows-mechanical-str-q22676917)", Chegg Study)

![alt text](https://d2vlcm61l7u1fs.cloudfront.net/media%2F1e6%2F1e67532c-1ca4-43b6-95bd-e0926e751b34%2Fphpz020E9.png "Truss")

모든 각은 45도 이다.<br>
All angles are 45 degrees.

각 마디에서의 힘의 평형은 다음과 같다.<br>
Force equilibrium equations at respective nodes are as follows.

![alt text](https://d2vlcm61l7u1fs.cloudfront.net/media%2Fff1%2Fff175bd1-3c4c-45bb-a997-30340bb30d1f%2FphpYdenAX.png "Equations")

미지수 17개를 모두 써서 표시하면 다음과 같을 것이다.<br>
Using all 17 unknowns, the equations would be as follows.

$$
\begin{align}
    -\alpha \cdot f_1+0 \cdot f_2 + 0 \cdot f_3 + 1 \cdot f_4 + \alpha \cdot f_5 + 0 \cdot f_6 + 0 \cdot f_7 + 0 \cdot f_8 + 0 \cdot f_9 + 0 \cdot f_{10} + 0 \cdot f_{11} + 0 \cdot f_{12} + 0 \cdot f_{13} + 0 \cdot f_{14} + 0 \cdot f_{15} + 0 \cdot f_{16} + 0 \cdot f_{17}=0
\end{align}
$$